## generate_data.py

In [1]:
import csv
import random
from datetime import datetime, timedelta
import json

# Generate data for 50 characters
NUM_ROWS = 1000

# Create the CSV file
OUTPUT_FILE = "troop_movements.csv"


def choose_a_side(home_world):
    """
    Randomly choose an empire or resistance side based on the likelihood 
    that someone from that world would join the rebel alliance.
    Args:
        home_world (dict): The home world data for the character.
    Returns:
        str: The empire or resistance side.
    """
    if home_world["rebel_likelihood"] > random.random():
        return "resistance"
    else:
        return "empire"


# Load home world data from JSON file
with open("home_worlds.json") as json_file:
    home_worlds = json.load(json_file)

# Generate data rows
data_rows = []
for i in range(1, NUM_ROWS + 1):
    # Generate random values for each column
    timestamp = datetime.now() - timedelta(seconds=i)
    unit_id = i
    unit_type = random.choice(
        ["stormtrooper", "tie_fighter", "at-st", "x-wing",
            "resistance_soldier", "at-at", "tie_silencer", "unknown"]
    )
    location_x = random.randint(1, 10)
    location_y = random.randint(1, 10)
    destination_x = random.randint(1, 10)
    destination_y = random.randint(1, 10)

    # Select a random home world from the available options
    home_world = random.choice(home_worlds)
    home_world_name = home_world["name"]
    empire_or_resistance = choose_a_side(home_world)

    # Create the data row
    data_row = [
        timestamp.strftime("%Y-%m-%d %H:%M:%S"),
        unit_id,
        unit_type,
        empire_or_resistance,
        location_x,
        location_y,
        destination_x,
        destination_y,
        home_world_name,
    ]

    # Add the data row to the list
    data_rows.append(data_row)

# Write the data to the CSV file
with open(OUTPUT_FILE, "w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(
        ["timestamp", "unit_id", "unit_type", "empire_or_resistance", "location_x", "location_y", "destination_x",
         "destination_y", "homeworld"]
    )
    writer.writerows(data_rows)

print("Data generation complete.")


Data generation complete.


### Display troop_movements.csv as DataFrame

In [2]:
import pandas as pd
df = pd.read_csv("troop_movements.csv")
print(df)

               timestamp  unit_id           unit_type empire_or_resistance  \
0    2024-02-07 12:19:28        1               at-st               empire   
1    2024-02-07 12:19:27        2             unknown           resistance   
2    2024-02-07 12:19:26        3         tie_fighter           resistance   
3    2024-02-07 12:19:25        4        tie_silencer               empire   
4    2024-02-07 12:19:24        5              x-wing           resistance   
..                   ...      ...                 ...                  ...   
995  2024-02-07 12:02:53      996         tie_fighter               empire   
996  2024-02-07 12:02:52      997               at-st           resistance   
997  2024-02-07 12:02:51      998             unknown           resistance   
998  2024-02-07 12:02:50      999  resistance_soldier               empire   
999  2024-02-07 12:02:49     1000             unknown           resistance   

     location_x  location_y  destination_x  destination_y   hom

### Part 2: Build a prediction model

In [3]:
# Create grouped data showing counts of empire vs resistance.
er = df.groupby("empire_or_resistance").size().reset_index(name="count")
print(er.head())

  empire_or_resistance  count
0               empire    480
1           resistance    520


In [4]:
# Create grouped data showing counts of characters by homeworld
homeworld_counts = df.groupby("homeworld").size().reset_index(name="count")
print(homeworld_counts.head())


     homeworld  count
0     Alderaan     25
1  Aleen Minor     26
2   Bestine IV     19
3        Cerea     15
4     Champala     26


In [5]:
# Created grouped data showing counts of characters by unit_type
unit_type_counts = df.groupby("unit_type").size().reset_index(name="count")
print(unit_type_counts.head())


            unit_type  count
0               at-at    128
1               at-st    130
2  resistance_soldier    116
3        stormtrooper    118
4         tie_fighter    119


In [6]:
# Engineer a new feature called is_resistance with a True or False value based on empire_or_resistance
df['is_resistance'] = df["empire_or_resistance"].map(lambda x: x == "resistance")
df.head()

,timestamp,unit_id,unit_type,empire_or_resistance,location_x,location_y,destination_x,destination_y,homeworld,is_resistance
0,2024-02-07 12:19:28,1,at-st,empire,4,8,1,3,Vulpter,False
1,2024-02-07 12:19:27,2,unknown,resistance,9,4,6,8,Champala,True
2,2024-02-07 12:19:26,3,tie_fighter,resistance,3,4,2,1,Bestine IV,True
3,2024-02-07 12:19:25,4,tie_silencer,empire,4,7,6,6,Corellia,False
4,2024-02-07 12:19:24,5,x-wing,resistance,6,5,9,4,Bestine IV,True


In [7]:
# Create a bar plot using Seaborn showing Empire vs Resistance distribution
import seaborn as sns
import matplotlib.pyplot as plt

sns.barplot(x = 'empire_or_resistance',y='count',data = er)

<Axes: xlabel='empire_or_resistance', ylabel='count'>

In [ ]:
# Create a prediction model using sklearn.tree.DecisionTreeClassifier that predicts if a character is joining either the Empire or the Resistance based on their homeworld and unit_type.
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split

In [ ]:
features = ["homeworld","unit_type"]
# Convert categorical features to numeric using pd.get_dummies. 
df1 = df.copy()
X_encoded = pd.get_dummies(df[features])
y_encoded = pd.get_dummies(df['empire_or_resistance'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y_encoded, test_size=0.2, random_state=1)

model = DecisionTreeClassifier()
model = model.fit(X_train,y_train)

y_pred = model.predict(X_test)
print(y_pred)

In [ ]:
# Get feature importances
importances = model.feature_importances_

# Create a DataFrame to hold the feature importances
feature_importances = pd.DataFrame({'Feature': X_encoded.columns, 'Importance': importances})
print(feature_importances)
feature_importances = feature_importances.sort_values('Importance', ascending=True)

# Create a bar plot that shows feature importance.
plt.figure(figsize=(8, 6))
plt.bar(feature_importances['Feature'], feature_importances['Importance'])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()

In [ ]:
# Save to model as a pickle file named trained_model.pkl
import pickle
filename = "trained_model.pkl"
with open(filename, 'wb') as file:
      pickle.dump(model, file)

### Part 3: Use the trained model with “real” data

Load data from troop_movements10m.csv (see Guided Project zip for file). This file contains 10 million records to be predicted. 
This data must be cleaned up a bit before it can be used:

⦁	Some unit_type records have a value of invalid_unit. Replace that with unknown.

⦁	Some location_x and location_Y values are missing. Use the ffill method to fill.

⦁	Save the clean data into a Parquet file named troop_movements10m.parquet.

⦁	You need to install pyarrow and fastparquet to support saving to a Parquet file.

pip install pyarrow

pip install fastparquet

Load the pickled model and load the data from the Parquet file into a data frame. Run the data through the model.


In [ ]:
df_trained = pd.read_csv('troop_movements.csv')

df_trained.head()

In [ ]:
%pip install pyarrow
%pip install fastparquet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
df_trained.unit_type.replace("invalid_unit","unknown")

0                   at-st
1                   at-at
2            stormtrooper
3                   at-st
4                 unknown
              ...        
995                 at-st
996    resistance_soldier
997          stormtrooper
998                 at-at
999    resistance_soldier
Name: unit_type, Length: 1000, dtype: object

In [ ]:
df_trained.location_x.ffill()
df_trained.location_y.ffill()


0      4
1      8
2      2
3      5
4      2
      ..
995    1
996    9
997    8
998    8
999    5
Name: location_y, Length: 1000, dtype: int64

In [ ]:
pquet = df_trained.to_parquet("clean_troopers.parquet")

In [ ]:
with open("trained_model.pkl",'rb') as f: 
    model2 = pickle.load(f)

parquet_db = pd.read_parquet('clean_troopers.parquet', engine = 'pyarrow')

In [ ]:
features = ["homeworld","unit_type"]
# Convert categorical features to numeric using pd.get_dummies. 
X_encoded = pd.get_dummies(parquet_db[features])
y_encoded = pd.get_dummies(parquet_db['empire_or_resistance'])

print(X_encoded)
print(y_encoded)

     homeworld_Alderaan  homeworld_Aleen Minor  homeworld_Bestine IV  \
0                     0                      0                     0   
1                     0                      0                     0   
2                     0                      0                     0   
3                     0                      0                     0   
4                     0                      0                     0   
..                  ...                    ...                   ...   
995                   0                      0                     0   
996                   0                      0                     0   
997                   0                      0                     0   
998                   0                      0                     0   
999                   0                      0                     0   

     homeworld_Cerea  homeworld_Champala  homeworld_Chandrila  \
0                  0                   0                    0   
1    

In [ ]:
print(X_encoded.head())

   homeworld_Alderaan  homeworld_Aleen Minor  homeworld_Bestine IV  \
0                   0                      0                     0   
1                   0                      0                     0   
2                   0                      0                     0   
3                   0                      0                     0   
4                   0                      0                     0   

   homeworld_Cerea  homeworld_Champala  homeworld_Chandrila  \
0                0                   0                    0   
1                0                   0                    0   
2                0                   0                    0   
3                0                   0                    0   
4                0                   0                    0   

   homeworld_Concord Dawn  homeworld_Corellia  homeworld_Dagobah  \
0                       0                   0                  0   
1                       0                   0                  0

In [ ]:
ypred = model2.predict(X_encoded)
print(ypred)

df_trained['Empire_Affilliated'] = ypred.map(lambda x: x[0] == 1 )

#df["empire_or_resistance"].map(lambda x: x == "resistance")
df_trained


[[1 0]
 [1 0]
 [0 1]
 ...
 [0 1]
 [0 1]
 [1 0]]


AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [ ]:
importances = model2.feature_importances_

# Create a DataFrame to hold the feature importances
feature_importances = pd.DataFrame({'Feature': X_encoded.columns, 'Importance': importances})
print(feature_importances)
feature_importances = feature_importances.sort_values('Importance', ascending=True)

# Create a bar plot that shows feature importance.
plt.figure(figsize=(8, 6))
plt.bar(feature_importances['Feature'], feature_importances['Importance'])
plt.xticks(rotation=90)
plt.xlabel('Feature')
plt.ylabel('Importance')
plt.title('Feature Importances')
plt.show()